In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import random
import collections
# use multiprocessing for multicore ops
import copy
#nltk.download()
from nltk.corpus import stopwords # Import the stop word list
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
from nltk import word_tokenize, pos_tag

def noun_count(q1, q2):
    s1 = pos_tag(nltk.word_tokenize(q1))
    s2 = pos_tag(nltk.word_tokenize(q2))
    n1 = []
    n2 = []
    for i,j in s1:
        if j == 'NN':
            n1.append(i)

    for i,j in s2:
        if j == 'NN':
            n2.append(i)

    s11= set(n1)
    s22 = set(n2)
    return len(s11.intersection(s22))
    


In [3]:
def verb_count(q1, q2):
    s1 = pos_tag(nltk.word_tokenize(q1))
    s2 = pos_tag(nltk.word_tokenize(q2))
    n1 = []
    n2 = []
    for i,j in s1:
        if j == 'VB':
            n1.append(i)

    for i,j in s2:
        if j == 'VB':
            n2.append(i)

    s11= set(n1)
    s22 = set(n2)
    return len(s11.intersection(s22))


In [4]:
def getSimilarity(df, cv):
    r,c = df.shape
    #print(df)
    #print(str(r)+ str(c))
    final_vec = []
    for j in range(r):
        a = cv.transform([df.iloc[j].question1])
        b = cv.transform([df.iloc[j].question2])
        sim = round(cosine_similarity(a,b).ravel()[0], 3)
        nc = noun_count(df.iloc[j].question1,df.iloc[j].question2)
        #vc = verb_count(df.iloc[j].question1,df.iloc[j].question2)
        #print(nc)
        c = df.iloc[j].is_duplicate
        final_vec.append([sim, nc, c])
    return final_vec


In [5]:
def question_to_words(raw_quest, lower=True, remove_not=True):
    lemmetizer = nltk.WordNetLemmatizer()

    letters_only = re.sub("[^a-zA-Z]", " ", raw_quest)
    if lower == True:
        letters_only = letters_only.lower()
    words = nltk.word_tokenize(letters_only)
    # words = nltk.word_tokenize(s)
    stops = set(stopwords.words("english"))
    not_stop = ['no', 'nor', 'not', "don't", "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't",
                'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn',
                "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
                "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    if remove_not == True:
        # stop = stops.difference(not_stop)
        stops = ['the', 'a', 'an', 'and', 'but', 'if', 'or', 'because', 'as', 'what', 'which', 'this', 'that', 'these',
                 'those', 'then',
                 'just', 'so', 'than', 'such', 'both', 'through', 'about', 'for', 'is', 'of', 'while', 'during', 'to',
                 'What', 'Which',
                 'Is', 'If', 'While', 'This']
    meaningful_words = [w for w in words if not w in stops]

    meaningful_words = [lemmetizer.lemmatize(w) for w in meaningful_words]
    return (" ".join(meaningful_words))


In [45]:
from nltk.stem import SnowballStemmer
import re
from string import punctuation

stop_words = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']

def text_to_wordlist(text, remove_stop_words=True, stem_words=False):
    # Clean the text, with the option to remove stop_words and to stem words.

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"What's", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r"switzerland", "Switzerland", text)
    text = re.sub(r"china", "China", text)
    text = re.sub(r"chinese", "Chinese", text) 
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r"quora", "Quora", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text) 
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iPhone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"gps", "GPS", text)
    text = re.sub(r"gst", "GST", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the US", "America", text)
    text = re.sub(r"Astrology", "astrology", text)
    text = re.sub(r"Method", "method", text)
    text = re.sub(r"Find", "find", text) 
    text = re.sub(r"banglore", "Banglore", text)
    text = re.sub(r" J K ", " JK ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [46]:
def get_cosine_df(X_train, y_train):
    X_train['question1'] = X_train['question1'].apply(text_to_wordlist)
    X_train['question2'] = X_train['question2'].apply(text_to_wordlist)
    #print(X_train.shape)
    #print(X_train.head)

    #print("Creating the bag of words...\n")

    vectorizer = CountVectorizer(analyzer="word", \
                                 tokenizer=None, \
                                 preprocessor=None, \
                                 stop_words=None, \
                                 ngram_range=(1,2),\
                                 max_features=5000)
    corpus = X_train["question1"] + " " + X_train["question2"]
    cv = vectorizer.fit(corpus)

    # cv
    #print(X_train)
    #print(y_train)
    new_data = pd.concat([X_train, y_train], axis=1, join_axes=[X_train.index])
    #print(new_data.head)
    sim = getSimilarity(new_data, cv)
    return sim

In [47]:

data2 = pd.read_csv("data/train.csv", delimiter=',', encoding="utf-8-sig")
data1 = copy.deepcopy(data2[:50000])
data1.dropna(inplace=True)
#print(data1.shape)
#print(data1.iloc[0:1, 3:])

X = data1[["question1", "question2"]]
y = data1[['is_duplicate']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#print(stopwords.words("english"))

#print(X_train.shape)

sim = get_cosine_df(X_train, y_train)
df1 = pd.DataFrame(sim)
#df1.columns = ["cos_sim","noun_count","verb_count", "y"]
df1.columns = ["cos_sim","noun_count", "y"]
#df1


#df1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
#X = df1[['cos_sim','noun_count', 'verb_count']]
X = df1[['cos_sim','noun_count']]
#print(X)
y = df1['y']
#X = X.values.reshape(-1, 1)
#y = y.values.reshape(-1,1)


In [49]:
# from sklearn.neighbors import KNeighborsClassifier
# neigh = KNeighborsClassifier(n_neighbors=3)
# neigh.fit(X,y)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [61]:
from sklearn.svm import SVC
clf_svm = SVC(gamma='auto')
clf_svm.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [62]:
print(X_test.shape)
sim = get_cosine_df(X_test, y_test)
df1 = pd.DataFrame(sim)
print(df1.shape)
#df1.columns = ["cos_sim","noun_count", 'verb_count',"y"]
df1.columns = ["cos_sim","noun_count","y"]
#df1
#print(df1.head)
Xt = df1[['cos_sim','noun_count',]]
yt = df1["y"]
#Xt = Xt.values.reshape(-1, 1)
p = neigh.predict(Xt)
# p1 = clf_svm.predict(Xt)

(16500, 2)
(16500, 3)


In [267]:
# p1

In [63]:
n = p.tolist()
print(len(n))
#n
# n1 = p1.tolist()
#print(len(n1))


16500


In [64]:
fact = yt.tolist()
print(len(fact))
#fact

16500


In [66]:
from sklearn.metrics import *

assert(len(fact)==len(p))
count = 0
for i in range(len(fact)):
    if fact[i]==p[i]:
        count+=1
print(count)
print("Accuracy = " + str(count/len(fact)) )

print("f1 " + str(f1_score(fact, p)))
print("accu " + str(accuracy_score(fact, p)))

10294
Accuracy = 0.6238787878787879
f1 0.5005633349428616
accu 0.6238787878787879


In [271]:
# assert(len(fact)==len(p1))
# count = 0
# for i in range(len(fact)):
#     if fact[i]==p1[i]:
#         count+=1
# print(count)
# print("Accuracy = " + str(count/len(fact)) )